In [0]:
x = 1
y = 4
sum = x + y
print(sum)

5


In [0]:
list = [1,2,3,4]
sum(list)

Out[2]: 10

Calculate Average

In [0]:
sum(list)/len(list)

Out[4]: 2.5

In [0]:
my_dict = {"name": "Alice", "age": 25}
print(my_dict["name"])   
my_dict["age"] = 26
print(my_dict.keys())
print(my_dict.values())

Alice
dict_keys(['name', 'age'])
dict_values(['Alice', 26])


In [0]:
my_set = {1,2,3,4,5}
my_set.add(6)
print(my_set)
my_set.update([2,7,8])
print(my_set)

{1, 2, 3, 4, 5, 6}
{1, 2, 3, 4, 5, 6, 7, 8}


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PythonDF").getOrCreate()

#Create DataFrame from list of tuples
data = [(1, "Alice"), (2, "Bob"), (3, "Carol")]
df = spark.createDataFrame(data, ["id", "name"])
df.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Carol|
+---+-----+



# Basic Transformations and Actions on DataFrames
- >    select() return specified columns
- >    filter() applies row-level conditions


   
 

In [0]:
# Select specific columns
df.select("name").show()

# Filter rows
df.filter(df["id"] > 1).show()

# Count rows
print(df.count())

# Describe summary statistics (numeric columns)
df.describe().show()

# Add new column with literal value
from pyspark.sql.functions import lit
df = df.withColumn("country", lit("India"))
df.show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
|Carol|
+-----+

+---+-----+
| id| name|
+---+-----+
|  2|  Bob|
|  3|Carol|
+---+-----+

3
+-------+---+-----+
|summary| id| name|
+-------+---+-----+
|  count|  3|    3|
|   mean|2.0| null|
| stddev|1.0| null|
|    min|  1|Alice|
|    max|  3|Carol|
+-------+---+-----+

+---+-----+-------+
| id| name|country|
+---+-----+-------+
|  1|Alice|  India|
|  2|  Bob|  India|
|  3|Carol|  India|
+---+-----+-------+



In [0]:
sales_data = [
    ("2024-01-01", "North", "Product A", 10, 200.0),
    ("2024-01-01", "South", "Product B", 5, 300.0),
    ("2024-01-02", "North", "Product A", 20, 400.0),
    ("2024-01-02", "South", "Product B", 10, 600.0),
    ("2024-01-03", "East",  "Product C", 15, 375.0),
]
columns = ["date", "region", "product", "quantity", "revenue"]
sales_df = spark.createDataFrame(sales_data, columns)
sales_df.show()

+----------+------+---------+--------+-------+
|      date|region|  product|quantity|revenue|
+----------+------+---------+--------+-------+
|2024-01-01| North|Product A|      10|  200.0|
|2024-01-01| South|Product B|       5|  300.0|
|2024-01-02| North|Product A|      20|  400.0|
|2024-01-02| South|Product B|      10|  600.0|
|2024-01-03|  East|Product C|      15|  375.0|
+----------+------+---------+--------+-------+



In [0]:
#Write a code for display total revenue for each product
from pyspark.sql.functions import sum
sales_df.groupBy("product").agg(sum("revenue")).show()

+---------+------------+
|  product|sum(revenue)|
+---------+------------+
|Product A|       600.0|
|Product B|       900.0|
|Product C|       375.0|
+---------+------------+



In [0]:
#Write a code for display total revenue for each product in visual table
from pyspark.sql.functions import sum
display(sales_df.groupBy("product").agg(sum("revenue")))

product,sum(revenue)
Product A,600.0
Product B,900.0
Product C,375.0


Databricks visualization. Run in Databricks to view.

# Average Revenue Per Region

In [0]:
from pyspark.sql.functions import max
sales_df.groupBy('region').agg(sum('revenue')/sum('quantity')).show()

+------+------------------------------+
|region|(sum(revenue) / sum(quantity))|
+------+------------------------------+
| North|                          20.0|
| South|                          60.0|
|  East|                          25.0|
+------+------------------------------+



In [0]:
#Display Region With Highest Revenue
from pyspark.sql.functions import max
sales_df.groupBy("region").agg(sum("revenue").alias("total_revenue")).orderBy("total_revenue",ascending=False).show(1)

+------+-------------+
|region|total_revenue|
+------+-------------+
| South|        900.0|
+------+-------------+
only showing top 1 row



# Average Revenue Per Product

In [0]:
from pyspark.sql. functions import avg
display(sales_df.groupBy("region").agg(avg("revenue").alias("avg_revenue_region")))

region,avg_revenue_region
North,300.0
South,450.0
East,375.0
